In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import transbigdata as tbd
import os
# display polars' progress bar.
# os.environ['POLARS_VERBOSE'] = '1'
import math
import multiprocessing
import json
import datetime

import csv
import torch

import CommonCode as cc

gParameters = cc.JSONConfig('./Parameters.json')
gGeoParameters = tbd.area_to_params(gParameters.get('gBoundsWuhan'), accuracy = 1000, method='rect')

# 生成武汉市的grid和对应mapping的表。
res = tbd.area_to_grid(gParameters.get('gBoundsWuhan'), accuracy = 1000)
wuhanRegion = res[0][['LONCOL', 'LATCOL']].copy()
wuhanRegion = wuhanRegion.apply(cc.GenerateGrid, lonColName='LONCOL', latColName='LATCOL', axis=1)
print(wuhanRegion.head(3))

def PoIGenerateGridMapping(PoIFeature,
                mappingColumnName = 'grid_mapping',
                mappingSavePath='./Data/Output/grid_mapping_wuhan.csv'):
    """_summary_
    这个函数的产生的原因是原始的grid分布十分不均匀，最小值 3622.0, 最大值 274546.0。
    而且28,200 行中有 23,102 行在40,000-60,000之间，分布十分不均匀。
    直接导致的后果是在使用nn.embedding的时候会导致词典数量过大，需要消耗过大内存（>24G）。所以必须做映射。
    
    输入是只包含grid的所有PoI区域的数据。
    Args:
        Tensor (torch.tensor): 输入的是只有grid一列的数据。
        mappingColumnName (str, optional): 在进行grid映射的时候需要指定一个新的列名. Defaults to 'grid_mapping'.
        mappingSavePath (str, optional): 映射关系静态化保存路径. Defaults to './Data/Output/grid_mapping.csv'.

    Returns:
        pandas.Dataframe: 返回包含 grid 和 grid_mapping 的所有停留点DataFrame。
    """
    GridColumnData = pd.DataFrame(PoIFeature.loc[:, 'grid'])
    # 生成列名。这一步不能删除，没有添加列名，就是一个series，后面就无法重新排序了。
    GridColumnData.columns = ['grid']
    # 去重之后共计9396个区域，数量没有变化。
    Grid_duplicated = GridColumnData.drop_duplicates()
    # 重新排序。查看前10个样本，感觉实际也没有做任何操作。可能已经排过序了。
    Grid_duplicated = Grid_duplicated.sort_values(by='grid', ascending=True)
    # 重置Index。
    Grid_duplicated = Grid_duplicated.reset_index(drop=True)
    # 将index 定义为新的映射列名。
    Grid_duplicated[mappingColumnName]= Grid_duplicated.index
    # 保留对应关系。将所有去重之后的已有停留点的包含grid的dataframe保留下来。
    Grid_duplicated.to_csv(mappingSavePath)
    # aftermapping = tensor_pd.merge(right=tensor_duplicated, on='grid', how='left').fillna(0)
    return Grid_duplicated
    
PoIGridMapping = PoIGenerateGridMapping(wuhanRegion)
print(PoIGridMapping.head(3))

In [ ]:
FolderPath = './Data/Geolif Trajectories 1.4/'
FilePath = f'./Data/Geolife Trajectories 1.4/1.csv'
# columns = ['ID', 'time', 'ID2', 'longitude', 'latitude', 'geohash', 'epoch0', 'epoch1']
SelectedColumne = ['ID', 'time', 'longitude', 'latitude', 'geohash']
# , nrows=10
df = pd.read_csv(FilePath, encoding='gb18030', header=0)

# 选取特定列。
df = df[SelectedColumne].copy()
# 将时间列转换为时间格式。
df['time'] = pd.to_datetime(df['time'])
# 按照时间排序。
df = df.sort_values(by='time')
# df = tbd.clean_outofbounds(df, bounds = gParameters.get('gBounds'), col = ['longitude', 'latitude'])
df['loncol'], df['latcol'] = tbd.GPS_to_grid(df['longitude'], df['latitude'], gGeoParameters)

stay, move = tbd.traj_stay_move(df, gGeoParameters, 
                                col=['ID', 'time', 'longitude', 'latitude'], 
                                activitytime=gParameters.get('gActivityTime'))
print(stay.shape, move.shape)
stay = stay.apply(cc.GenerateGrid, lonColName='LONCOL', latColName='LATCOL', axis=1)
print(stay.shape, move.shape)